In [1]:
import pandas as pd 
import numpy as np 
import sqlite3

In [2]:
##creating a new sqllite database
conn = sqlite3.connect('oklahoma_homes_sale.db')

In [3]:
query3=("""\
SELECT * FROM homes
               """)

In [4]:
new1=pd.read_sql(query3, conn)

In [5]:
new1.shape

(8780, 10)

In [6]:
##use this to see if update strategy worked##figureout why some of these houses are showing wrong prices 
new1[new1['unique_id']=='aab29e962']

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id
2477,"2532SOwassoAvenue,Tulsa,OK74114",749000.0,74114,Tulsa,4.0,3.0,2603.0,0.186,Residential,aab29e962


In [7]:
new1.head()

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id
0,"3101N.E,50,OklahomaCity,OK73121",1200000.0,73121,50,5.0,5.1,4834.0,0.304,Single-FamilyHome,528db3a24
1,"20685E48thStreetS,BrokenArrow,OK74014",265000.0,74014,Broken Arrow,3.0,2.0,1960.0,0.172,Residential,5b7c45cb9
2,"18521FelizDrive,OklahomaCity,OK73012",499999.0,73012,Oklahoma City,4.0,2.5,3141.0,0.304,Single-FamilyHome,efeaa8beb
3,"2010SECypressCircle,Bartlesville,OK74006",237990.0,74006,Bartlesville,3.0,2.0,1412.0,0.202,Residential,5dc223a37
4,"1704NWOakAve,Lawton,OK73507",107500.0,73507,Lawton,3.0,2.0,1100.0,0.304,Single-FamilyHome,55d5a6eb6


In [8]:
## replacing properties with sq_ft less than 50 with the median sqft 

In [9]:
def sq_ft(x):
    if x<50:
        return np.nan
    else: 
        return x 

In [10]:
new1['sq_ft']=new1['sq_ft'].apply(sq_ft)

In [11]:
new1['sq_ft']=new1['sq_ft'].fillna(new1['sq_ft'].median())

In [12]:
##removing spaces from city to allow for filtering. 
new1['city']=new1['city'].str.strip()

In [13]:
new1['price_per_sq_ft']=new1['price']/new1['sq_ft']

In [14]:
new1.head()

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft
0,"3101N.E,50,OklahomaCity,OK73121",1200000.0,73121,50,5.0,5.1,4834.0,0.304,Single-FamilyHome,528db3a24,248.241622
1,"20685E48thStreetS,BrokenArrow,OK74014",265000.0,74014,Broken Arrow,3.0,2.0,1960.0,0.172,Residential,5b7c45cb9,135.204082
2,"18521FelizDrive,OklahomaCity,OK73012",499999.0,73012,Oklahoma City,4.0,2.5,3141.0,0.304,Single-FamilyHome,efeaa8beb,159.184655
3,"2010SECypressCircle,Bartlesville,OK74006",237990.0,74006,Bartlesville,3.0,2.0,1412.0,0.202,Residential,5dc223a37,168.548159
4,"1704NWOakAve,Lawton,OK73507",107500.0,73507,Lawton,3.0,2.0,1100.0,0.304,Single-FamilyHome,55d5a6eb6,97.727273


In [15]:
##getting rid of the properties that had wierd prices associated with them  
a=new1['price']>100000
b=new1['price_per_sq_ft']>30

In [16]:
new2=new1[a&b]

In [17]:
##new3[new3['price_per_sq_ft']<60].sort_values('price_per_sq_ft',ascending=False)

In [18]:
new2[new2['unique_id'].isin(['34f64cdc8','cc78ccfde','77ff34515','e2099206e','14c5ebcf8'])]

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft


In [19]:
#new2=new2.drop(['price_per_sq_ft'],axis=1)

In [20]:
#devon tower zipcode
##73102

In [21]:
#calculating distance between two zipcodes
import pgeocode
#Geographical distance is the distance measured along the surface of the earth. 
#The formulae  calculate distances between points which are defined by geographical coordinates in 
#terms of latitude and longitude. This distance is an element in solving the second (inverse) geodetic problem.
dist=pgeocode.GeoDistance('US')
#adding OU medical center distance to our datafame 
new2['devon_zip']='73102'
new2['geo_distance_km']=dist.query_postal_code(new2['zip_code'].to_list(),new2['devon_zip'].to_list())

<ipython-input-21-0ba4f7c071fb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new2['devon_zip']='73102'
<ipython-input-21-0ba4f7c071fb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new2['geo_distance_km']=dist.query_postal_code(new2['zip_code'].to_list(),new2['devon_zip'].to_list())


In [22]:
new2.head()

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft,devon_zip,geo_distance_km
0,"3101N.E,50,OklahomaCity,OK73121",1200000.0,73121,50,5.0,5.1,4834.0,0.304,Single-FamilyHome,528db3a24,248.241622,73102,7.726523
1,"20685E48thStreetS,BrokenArrow,OK74014",265000.0,74014,Broken Arrow,3.0,2.0,1960.0,0.172,Residential,5b7c45cb9,135.204082,73102,174.615563
2,"18521FelizDrive,OklahomaCity,OK73012",499999.0,73012,Oklahoma City,4.0,2.5,3141.0,0.304,Single-FamilyHome,efeaa8beb,159.184655,73102,17.085296
3,"2010SECypressCircle,Bartlesville,OK74006",237990.0,74006,Bartlesville,3.0,2.0,1412.0,0.202,Residential,5dc223a37,168.548159,73102,200.698157
4,"1704NWOakAve,Lawton,OK73507",107500.0,73507,Lawton,3.0,2.0,1100.0,0.304,Single-FamilyHome,55d5a6eb6,97.727273,73102,123.115383


In [23]:
##bedroom to bathroom ratio 
new2['bed_to_bath_ratio']=new2['bath_rooms']/new2['bed_rooms']

<ipython-input-23-a9e44047c89b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new2['bed_to_bath_ratio']=new2['bath_rooms']/new2['bed_rooms']


In [24]:
#adding average price by zipcode 
#average_price_byzipcode=new2.groupby('zip_code')[['price']].mean().reset_index().rename({'price':'average_homeprice_byzip'},axis=1)

In [25]:
#adding average price by zipcode 
median_price_byzipcode=new2.groupby('zip_code')[['price']].median().reset_index().rename({'price':'median_homeprice_byzip'},axis=1)

In [26]:
median_price_byzipcode

,zip_code,median_homeprice_byzip
0,73003,275000.0
1,73003,295500.0
2,73005,189900.0
3,73006,240000.0
4,73006,172700.0
...,...,...
545,74964,175000.0
546,74965,139000.0
547,74965,129000.0
548,74966,425000.0


In [27]:
new2['zip_code']=new2['zip_code'].str.replace(' ','')

<ipython-input-27-7d2f9c14d852>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new2['zip_code']=new2['zip_code'].str.replace(' ','')


In [28]:
#adding census zipcode data
zip_census=pd.read_excel(r'C:\Users\arnie\Desktop\OU health stuff\zip_stats.xlsx')

In [29]:
zip_census.columns=zip_census.columns.str.replace(' ', '')

In [30]:
zip_census.head()

,AddressZipCode,Median_income,Mean_income,Population
0,1001,56662.5735,66687.8,16445
1,1002,49853.4177,75062.6,28069
2,1003,28462.0000,35121,8491
3,1005,75423.0000,82442,4798
4,1007,79076.3540,85802,12962


In [31]:
#zip_census=zip_census.drop('Median_income',axis=1)

In [32]:
zip_census=zip_census.drop('Mean_income',axis=1)

In [33]:
zip_census=zip_census.rename({'AddressZipCode':'zip_code'},axis=1)

In [34]:
##merging 
new3=new2.merge(median_price_byzipcode,on='zip_code',how='left')

In [35]:
zip_census['zip_code'].astype(str)

0         1001
1         1002
2         1003
3         1005
4         1007
         ...  
32629    99923
32630    99925
32631    99926
32632    99927
32633    99929
Name: zip_code, Length: 32634, dtype: object

In [36]:
zip_census['zip_code']=zip_census['zip_code'].astype(str)

In [37]:
new4=new3.merge(zip_census,on='zip_code',how='left')

In [38]:
new4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7449 entries, 0 to 7448
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   address                 7449 non-null   object 
 1   price                   7449 non-null   float64
 2   zip_code                7428 non-null   object 
 3   city                    7447 non-null   object 
 4   bed_rooms               7449 non-null   float64
 5   bath_rooms              7449 non-null   float64
 6   sq_ft                   7449 non-null   float64
 7   lot_size                7449 non-null   float64
 8   property_type           7449 non-null   object 
 9   unique_id               7449 non-null   object 
 10  price_per_sq_ft         7449 non-null   float64
 11  devon_zip               7449 non-null   object 
 12  geo_distance_km         3611 non-null   float64
 13  bed_to_bath_ratio       7449 non-null   float64
 14  median_homeprice_byzip  7349 non-null   

In [39]:
new4.columns

Index(['address', 'price', 'zip_code', 'city', 'bed_rooms', 'bath_rooms',
       'sq_ft', 'lot_size', 'property_type', 'unique_id', 'price_per_sq_ft',
       'devon_zip', 'geo_distance_km', 'bed_to_bath_ratio',
       'median_homeprice_byzip', 'Median_income', 'Population'],
      dtype='object')

In [40]:
#new4['Mean_income']=new4['Mean_income'].astype('float')

In [41]:
new4.head()

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft,devon_zip,geo_distance_km,bed_to_bath_ratio,median_homeprice_byzip,Median_income,Population
0,"3101N.E,50,OklahomaCity,OK73121",1200000.0,73121,50,5.0,5.1,4834.0,0.304,Single-FamilyHome,528db3a24,248.241622,73102,7.726523,1.020000,475000.0,49578.9497,3159.0
1,"20685E48thStreetS,BrokenArrow,OK74014",265000.0,74014,Broken Arrow,3.0,2.0,1960.0,0.172,Residential,5b7c45cb9,135.204082,73102,174.615563,0.666667,269900.0,67259.7541,22568.0
2,"18521FelizDrive,OklahomaCity,OK73012",499999.0,73012,Oklahoma City,4.0,2.5,3141.0,0.304,Single-FamilyHome,efeaa8beb,159.184655,73102,17.085296,0.625000,303970.0,86655.9436,13373.0
3,"2010SECypressCircle,Bartlesville,OK74006",237990.0,74006,Bartlesville,3.0,2.0,1412.0,0.202,Residential,5dc223a37,168.548159,73102,200.698157,0.666667,246990.0,55052.9411,23892.0
4,"1704NWOakAve,Lawton,OK73507",107500.0,73507,Lawton,3.0,2.0,1100.0,0.304,Single-FamilyHome,55d5a6eb6,97.727273,73102,123.115383,0.666667,159000.0,45882.9925,21585.0


In [42]:
#new4['percapitaveragezipprice']=new4['average_homeprice_byzip']/new4['Population']

In [43]:
new4['percapitmedianzipprice']=new4['median_homeprice_byzip']/new4['Population']

In [44]:
new4.corr()['price'].sort_values(ascending=False)

price                     1.000000
sq_ft                     0.685816
bath_rooms                0.616898
bed_to_bath_ratio         0.437117
bed_rooms                 0.385851
median_homeprice_byzip    0.354435
price_per_sq_ft           0.250448
Median_income             0.170854
lot_size                  0.160745
percapitmedianzipprice    0.140648
geo_distance_km          -0.004990
Population               -0.067127
Name: price, dtype: float64

In [45]:
##trying to add distance to down town using geopy 

In [46]:
##tomorrow start by building a predictive model and analyze furtheref

In [47]:
pd.options.display.float_format = '{:20,.2f}'.format

In [48]:
new4.head()

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft,devon_zip,geo_distance_km,bed_to_bath_ratio,median_homeprice_byzip,Median_income,Population,percapitmedianzipprice
0,"3101N.E,50,OklahomaCity,OK73121","1,200,000.00",73121,50,5.00,5.10,"4,834.00",0.30,Single-FamilyHome,528db3a24,248.24,73102,7.73,1.02,"475,000.00","49,578.95","3,159.00",150.36
1,"20685E48thStreetS,BrokenArrow,OK74014","265,000.00",74014,Broken Arrow,3.00,2.00,"1,960.00",0.17,Residential,5b7c45cb9,135.20,73102,174.62,0.67,"269,900.00","67,259.75","22,568.00",11.96
2,"18521FelizDrive,OklahomaCity,OK73012","499,999.00",73012,Oklahoma City,4.00,2.50,"3,141.00",0.30,Single-FamilyHome,efeaa8beb,159.18,73102,17.09,0.62,"303,970.00","86,655.94","13,373.00",22.73
3,"2010SECypressCircle,Bartlesville,OK74006","237,990.00",74006,Bartlesville,3.00,2.00,"1,412.00",0.20,Residential,5dc223a37,168.55,73102,200.70,0.67,"246,990.00","55,052.94","23,892.00",10.34
4,"1704NWOakAve,Lawton,OK73507","107,500.00",73507,Lawton,3.00,2.00,"1,100.00",0.30,Single-FamilyHome,55d5a6eb6,97.73,73102,123.12,0.67,"159,000.00","45,882.99","21,585.00",7.37


In [49]:
new4[new4['city']=='Edmond'].sort_index(ascending=False).head(30)

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft,devon_zip,geo_distance_km,bed_to_bath_ratio,median_homeprice_byzip,Median_income,Population,percapitmedianzipprice
7324,"349MemoryLane,Edmond,OK73013","214,000.00",73013,Edmond,3.00,2.00,"1,363.00",0.21,Residential,2506e2dd8,157.01,73102,nan,0.67,"340,000.00","72,051.32","31,732.00",10.71
7323,"117StonebrookWay,Edmond,OK73003","294,000.00",73003,Edmond,3.00,2.00,"1,995.00",0.16,Residential,0b9353b30,147.37,73102,nan,0.67,"275,000.00","54,814.30","16,601.00",16.57
7322,"1308EchohollowTrail,Edmond,OK73025","499,900.00",73025,Edmond,4.00,3.00,"3,187.00",0.35,Residential,36c49acb6,156.86,73102,nan,0.75,"489,000.00","99,918.64","6,543.00",74.74
7321,"2821MorganTrail,Edmond,OK73012","249,500.00",73012,Edmond,3.00,2.00,"1,630.00",0.17,Residential,373c3b01c,153.07,73102,nan,0.67,"303,970.00","86,655.94","13,373.00",22.73
7320,"1905WillowBendDrive,Edmond,OK73003","312,500.00",73003,Edmond,4.00,2.00,"2,121.00",0.27,Residential,169223a0e,147.34,73102,nan,0.50,"275,000.00","54,814.30","16,601.00",16.57
7319,"2612RedwoodLane,Edmond,OK73013","185,000.00",73013,Edmond,3.00,2.00,"1,531.00",0.16,Residential,97b3ed8cc,120.84,73102,nan,0.67,"340,000.00","72,051.32","31,732.00",10.71
7318,"101AmberwoodRoad,Edmond,OK73003","349,900.00",73003,Edmond,4.00,3.00,"2,230.00",0.31,Residential,c75a86652,156.91,73102,nan,0.75,"275,000.00","54,814.30","16,601.00",16.57
7317,"3317SandPlumLane,Edmond,OK73003","379,000.00",73003,Edmond,3.00,3.00,"2,547.00",0.21,Residential,1a3583606,148.80,73102,nan,1.00,"275,000.00","54,814.30","16,601.00",16.57
7316,"4825ChamplainPlace,Edmond,OK73025","399,900.00",73025,Edmond,4.00,3.00,"2,438.00",0.24,Residential,f91239c20,164.03,73102,nan,0.75,"489,000.00","99,918.64","6,543.00",74.74
7315,"1405OldeNorthPlace,Edmond,OK73034","350,000.00",73034,Edmond,4.00,3.00,"2,713.00",0.25,Residential,20114558e,129.01,73102,nan,0.75,"438,340.00","71,515.79","31,219.00",14.04


In [50]:
new4[new4['address']=='2433RivenRockRoad,Edmond,OK73034']

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft,devon_zip,geo_distance_km,bed_to_bath_ratio,median_homeprice_byzip,Median_income,Population,percapitmedianzipprice
3232,"2433RivenRockRoad,Edmond,OK73034","405,000.00",73034,Edmond,3.00,4.00,"2,255.00",0.17,Residential,551b68a41,179.60,73102,21.86,1.33,"438,340.00","71,515.79","31,219.00",14.04


In [51]:
new4.groupby('city')[['price']].mean().sort_values(by='price',ascending=False).head(30)

,price
city,
Wilburton,"4,200,000.00"
Marble City,"3,250,000.00"
Caddo,"2,200,000.00"
Bokchito,"2,200,000.00"
Wayne,"2,000,000.00"
Wister,"1,403,333.33"
Crescent,"1,390,000.00"
50,"1,200,000.00"
Antlers,"1,180,600.00"


In [52]:
new4[new4['city']=='Big Cabin']

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft,devon_zip,geo_distance_km,bed_to_bath_ratio,median_homeprice_byzip,Median_income,Population,percapitmedianzipprice
68,"13492N430Road,BigCabin,OK74332","925,000.00",74332,Big Cabin,3.00,2.00,"2,231.00",0.30,Residential,650217006,414.61,73102,237.81,0.67,"555,000.00","47,074.85","1,810.00",306.63
3616,"27250E340Road,BigCabin,OK74332","185,000.00",74332,Big Cabin,2.00,1.00,720.00,10.00,Residential,cba8c869c,256.94,73102,237.81,0.50,"555,000.00","47,074.85","1,810.00",306.63
6777,"1305s4305rd,BigCabin,OK74332","300,000.00",74332,Big Cabin,3.00,2.00,"1,200.00",30.00,FarmAndAgriculture,58007e6d0,250.00,73102,nan,0.67,"555,000.00","47,074.85","1,810.00",306.63


In [53]:
new4.isna().sum()

address                      0
price                        0
zip_code                    21
city                         2
bed_rooms                    0
bath_rooms                   0
sq_ft                        0
lot_size                     0
property_type                0
unique_id                    0
price_per_sq_ft              0
devon_zip                    0
geo_distance_km           3838
bed_to_bath_ratio            0
median_homeprice_byzip     100
Median_income               22
Population                  22
percapitmedianzipprice     101
dtype: int64

In [54]:
##replacing missing values with the median 
new4['geo_distance_km']=new4['geo_distance_km'].fillna(new4['geo_distance_km'].mean())
new4['median_homeprice_byzip']=new4['median_homeprice_byzip'].fillna(new4['median_homeprice_byzip'].median())
new4['Median_income']=new4['Median_income'].fillna(new4['Median_income'].median())
new4['Population']=new4['Population'].fillna(new4['Population'].median())
new4['percapitmedianzipprice']=new4['percapitmedianzipprice'].fillna(new4['percapitmedianzipprice'].median())

In [55]:
##exporting to csv in order to use the medians and averages for new homes on sale 
#new4.to_csv('null.csv',index=False)

In [56]:
new5=new4.dropna()

In [57]:
new5.head()

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft,devon_zip,geo_distance_km,bed_to_bath_ratio,median_homeprice_byzip,Median_income,Population,percapitmedianzipprice
0,"3101N.E,50,OklahomaCity,OK73121","1,200,000.00",73121,50,5.00,5.10,"4,834.00",0.30,Single-FamilyHome,528db3a24,248.24,73102,7.73,1.02,"475,000.00","49,578.95","3,159.00",150.36
1,"20685E48thStreetS,BrokenArrow,OK74014","265,000.00",74014,Broken Arrow,3.00,2.00,"1,960.00",0.17,Residential,5b7c45cb9,135.20,73102,174.62,0.67,"269,900.00","67,259.75","22,568.00",11.96
2,"18521FelizDrive,OklahomaCity,OK73012","499,999.00",73012,Oklahoma City,4.00,2.50,"3,141.00",0.30,Single-FamilyHome,efeaa8beb,159.18,73102,17.09,0.62,"303,970.00","86,655.94","13,373.00",22.73
3,"2010SECypressCircle,Bartlesville,OK74006","237,990.00",74006,Bartlesville,3.00,2.00,"1,412.00",0.20,Residential,5dc223a37,168.55,73102,200.70,0.67,"246,990.00","55,052.94","23,892.00",10.34
4,"1704NWOakAve,Lawton,OK73507","107,500.00",73507,Lawton,3.00,2.00,"1,100.00",0.30,Single-FamilyHome,55d5a6eb6,97.73,73102,123.12,0.67,"159,000.00","45,882.99","21,585.00",7.37


In [58]:
new5['city'].value_counts().head(30)

Oklahoma City    1928
Tulsa            1054
Edmond            567
Norman            448
Broken Arrow      382
Lawton            259
Moore             224
Midwest City      149
Claremore         119
Owasso            101
Stillwater         98
Shawnee            96
Yukon              81
Bixby              69
Piedmont           66
Jenks              62
Elgin              61
Choctaw            60
Mustang            51
Del City           49
Bartlesville       46
Coweta             44
Collinsville       42
Sand Springs       41
Newcastle          40
Enid               40
Catoosa            35
El Reno            34
Broken Bow         34
McLoud             32
Name: city, dtype: int64

##categorical variables for city 
def city (x):
    if x=='Oklahoma City':
        return 'OKC'
    if x=='Tulsa':
        return 'Tulsa'
    if x=='Edmond':
        return 'Edmond'
    if x=='Norman':
        return 'Norman'
    if x=='Lawton':
        return 'Lawton'
    if x=='Broken Arrow':
        return 'BrokenArrow'
    if x=='Moore':
        return 'Moore'
    
    else:
        return 'other'

new5['city']=new5['city'].apply(city)

In [95]:
new5.head()

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft,devon_zip,geo_distance_km,bed_to_bath_ratio,median_homeprice_byzip,Median_income,Population,percapitmedianzipprice
0,"3101N.E,50,OklahomaCity,OK73121","1,200,000.00",73121,other,5.00,5.10,"4,834.00",0.30,Single-FamilyHome,528db3a24,248.24,73102,7.73,1.02,"475,000.00","49,578.95","3,159.00",150.36
1,"20685E48thStreetS,BrokenArrow,OK74014","265,000.00",74014,BrokenArrow,3.00,2.00,"1,960.00",0.17,Residential,5b7c45cb9,135.20,73102,174.62,0.67,"269,900.00","67,259.75","22,568.00",11.96
2,"18521FelizDrive,OklahomaCity,OK73012","499,999.00",73012,OKC,4.00,2.50,"3,141.00",0.30,Single-FamilyHome,efeaa8beb,159.18,73102,17.09,0.62,"303,970.00","86,655.94","13,373.00",22.73
3,"2010SECypressCircle,Bartlesville,OK74006","237,990.00",74006,other,3.00,2.00,"1,412.00",0.20,Residential,5dc223a37,168.55,73102,200.70,0.67,"246,990.00","55,052.94","23,892.00",10.34
4,"1704NWOakAve,Lawton,OK73507","107,500.00",73507,Lawton,3.00,2.00,"1,100.00",0.30,Single-FamilyHome,55d5a6eb6,97.73,73102,123.12,0.67,"159,000.00","45,882.99","21,585.00",7.37


In [96]:
new5['city'].value_counts().head(30)

other          2513
OKC            1895
Tulsa          1038
Edmond          556
Norman          444
BrokenArrow     373
Lawton          250
Moore           222
Name: city, dtype: int64

city_dum=pd.get_dummies(new5['city']).drop('other',axis=1)

new6=pd.concat([new5,city_dum],axis=1)

In [59]:
new6=new5.copy()

In [60]:
new6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7426 entries, 0 to 7448
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   address                 7426 non-null   object 
 1   price                   7426 non-null   float64
 2   zip_code                7426 non-null   object 
 3   city                    7426 non-null   object 
 4   bed_rooms               7426 non-null   float64
 5   bath_rooms              7426 non-null   float64
 6   sq_ft                   7426 non-null   float64
 7   lot_size                7426 non-null   float64
 8   property_type           7426 non-null   object 
 9   unique_id               7426 non-null   object 
 10  price_per_sq_ft         7426 non-null   float64
 11  devon_zip               7426 non-null   object 
 12  geo_distance_km         7426 non-null   float64
 13  bed_to_bath_ratio       7426 non-null   float64
 14  median_homeprice_byzip  7426 non-null   

In [61]:
new6.head()

,address,price,zip_code,city,bed_rooms,bath_rooms,sq_ft,lot_size,property_type,unique_id,price_per_sq_ft,devon_zip,geo_distance_km,bed_to_bath_ratio,median_homeprice_byzip,Median_income,Population,percapitmedianzipprice
0,"3101N.E,50,OklahomaCity,OK73121","1,200,000.00",73121,50,5.00,5.10,"4,834.00",0.30,Single-FamilyHome,528db3a24,248.24,73102,7.73,1.02,"475,000.00","49,578.95","3,159.00",150.36
1,"20685E48thStreetS,BrokenArrow,OK74014","265,000.00",74014,Broken Arrow,3.00,2.00,"1,960.00",0.17,Residential,5b7c45cb9,135.20,73102,174.62,0.67,"269,900.00","67,259.75","22,568.00",11.96
2,"18521FelizDrive,OklahomaCity,OK73012","499,999.00",73012,Oklahoma City,4.00,2.50,"3,141.00",0.30,Single-FamilyHome,efeaa8beb,159.18,73102,17.09,0.62,"303,970.00","86,655.94","13,373.00",22.73
3,"2010SECypressCircle,Bartlesville,OK74006","237,990.00",74006,Bartlesville,3.00,2.00,"1,412.00",0.20,Residential,5dc223a37,168.55,73102,200.70,0.67,"246,990.00","55,052.94","23,892.00",10.34
4,"1704NWOakAve,Lawton,OK73507","107,500.00",73507,Lawton,3.00,2.00,"1,100.00",0.30,Single-FamilyHome,55d5a6eb6,97.73,73102,123.12,0.67,"159,000.00","45,882.99","21,585.00",7.37


In [62]:
#excluding object datatypes 
new7=new6.set_index('unique_id').select_dtypes(exclude=['object'])

In [63]:
new7.head()

,price,bed_rooms,bath_rooms,sq_ft,lot_size,price_per_sq_ft,geo_distance_km,bed_to_bath_ratio,median_homeprice_byzip,Median_income,Population,percapitmedianzipprice
unique_id,,,,,,,,,,,,
528db3a24,"1,200,000.00",5.00,5.10,"4,834.00",0.30,248.24,7.73,1.02,"475,000.00","49,578.95","3,159.00",150.36
5b7c45cb9,"265,000.00",3.00,2.00,"1,960.00",0.17,135.20,174.62,0.67,"269,900.00","67,259.75","22,568.00",11.96
efeaa8beb,"499,999.00",4.00,2.50,"3,141.00",0.30,159.18,17.09,0.62,"303,970.00","86,655.94","13,373.00",22.73
5dc223a37,"237,990.00",3.00,2.00,"1,412.00",0.20,168.55,200.70,0.67,"246,990.00","55,052.94","23,892.00",10.34
55d5a6eb6,"107,500.00",3.00,2.00,"1,100.00",0.30,97.73,123.12,0.67,"159,000.00","45,882.99","21,585.00",7.37


In [64]:
##looking at only properties under 1,000,000
new8=new7[new7['price']<500000]

In [65]:
new8.shape

(6206, 12)

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
train_set, test_set= train_test_split (new8, test_size=.0000001, random_state=35)

In [68]:
X_train=train_set.drop('price',axis=1)

In [69]:
Y_train=train_set[['price']]

In [70]:
x_test=test_set.drop('price',axis=1)

In [71]:
y_test=test_set[['price']]

In [72]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [122]:
models = [
          ('LinearRegression', LinearRegression()), 
         # ('KNN', KNeighborsClassifier()),
          #('SVC', SVC()), 
          
          #('GNB', GaussianNB()),
          ('RandomForest', RandomForestRegressor()),
          ('XGB',xgboost.XGBRegressor())
        ]
for name, model in models:
    clf = model.fit(X_train,Y_train.values.ravel())
    y_pred = clf.predict(x_test)
    print(name)
    #print(classification_report(y_test, y_pred))
    print('RMSE:',np.sqrt(mean_squared_error(y_test,y_pred)))
    print('MAE:',(mean_absolute_error(y_test,y_pred)))
    #print(pd.DataFrame(confusion_matrix(y_test,y_pred)))

LinearRegression
RMSE: 9751.358862782829
MAE: 9751.358862782829
RandomForest
RMSE: 238.9899999999907
MAE: 238.9899999999907
XGB
RMSE: 4200.3125
MAE: 4200.3125


##looking at performance on training set
models = [
          ('LinearRegression', LinearRegression()), 
         # ('KNN', KNeighborsClassifier()),
          #('SVC', SVC()), 
          
          #('GNB', GaussianNB()),
          ('RandomForest', RandomForestRegressor()),
          ('XGB',xgboost.XGBRegressor())
        ]
for name, model in models:
    clf = model.fit(X_train,Y_train.values.ravel())
    y_pred = clf.predict(X_train)
    print(name)
    #print(classification_report(y_test, y_pred))
    print('RMSE:',np.sqrt(mean_squared_error(Y_train,y_pred)))
    print('MAE:',(mean_absolute_error(Y_train,y_pred)))
    #print(pd.DataFrame(confusion_matrix(y_test,y_pred)))

In [73]:
#sending to be used for filling nulls in new predictions 
new8.to_csv("null.csv",index=False)

In [74]:
##testing random forest 
rf=RandomForestRegressor()

In [75]:
rf.fit(X_train,Y_train.values.ravel())

RandomForestRegressor()

In [ ]:
xgb=xgboost.XGBRegressor()

In [ ]:
#xgb.fit(X_train,Y_train.values.ravel())

In [ ]:
#import shap

In [ ]:
#values=shap.TreeExplainer(xgb).shap_values(X_train)

In [ ]:
#shap.summary_plot(values, X_train)

In [ ]:
test_set['price_predictions']=rf.predict(x_test)

In [ ]:
test_set['diff']=test_set['price_predictions']-test_set['price']

In [ ]:
undervalued_sort=test_set.sort_values('diff',ascending=False).reset_index()

In [ ]:
undervalued_sort

In [ ]:
undervalued_sort2=undervalued_sort[['unique_id','price_predictions','diff']]

In [ ]:
see=undervalued_sort2.merge(new6,on='unique_id',how='inner')

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
see.head(20)

In [ ]:
see[see['unique_id'].isin(['d576d7cae','a7f1995dd','aa631fc06','00d8aa003'])]

In [76]:
##saving the winning model 
import joblib

In [77]:
##saving the winning model to a pkl. file  xgboost model 
joblib.dump(rf,'rf_final_2.pkl')

['rf_final_2.pkl']